In [1]:
from pth_methods import *
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [2]:
device = get_device()
device

device:cuda


'cuda'

# 读取数据

In [3]:
data = pd.read_csv('GSE25066_merge.csv')

In [4]:
data[data['group'] == 0].shape

(99, 13237)

In [5]:
data[data['group'] == 1].shape

(389, 13237)

In [6]:
df = data[data['group'] == 0]

In [7]:
group = df.pop('group')

In [8]:
setup_seed(42)

# AutoEncoder

In [21]:
# 分离训练集和测试集
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

# 转换为tensor
train_tensor = torch.tensor(train_data.values, dtype=torch.float32).to(device)
test_tensor = torch.tensor(test_data.values, dtype=torch.float32).to(device)

# 定义模型、损失函数和优化器
input_dim = 13236
encoding_dim = 512
GSE25066_AutoEncoder_Model = Autoencoder(input_dim, encoding_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(GSE25066_AutoEncoder_Model.parameters(), lr=0.0001)


train_loader = torch.utils.data.DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

## 训练AutoEncoder

In [22]:
train_tensor.shape

torch.Size([79, 13236])

In [35]:
batch_size = 64
min_loss = 0.001

In [36]:
min_now_loss = 10000
num=0;
while min_now_loss > min_loss:
    num += 1
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs = data
        data = data.to(device)
        optimizer.zero_grad()
        outputs = GSE25066_AutoEncoder_Model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        ae_loss = running_loss / len(train_loader)
        if (ae_loss < min_now_loss):
            min_now_loss = ae_loss
    if num % 1000 == 0: 
        print('Epoch [%d], Loss: %.4f' % (num, ae_loss))


Epoch [1000], Loss: 0.8694
Epoch [2000], Loss: 0.8677
Epoch [3000], Loss: 0.8718
Epoch [4000], Loss: 0.8641
Epoch [5000], Loss: 0.8665
Epoch [6000], Loss: 0.8679
Epoch [7000], Loss: 0.8660
Epoch [8000], Loss: 0.8666
Epoch [9000], Loss: 0.8665
Epoch [10000], Loss: 0.8675
Epoch [11000], Loss: 0.8660
Epoch [12000], Loss: 0.8694
Epoch [13000], Loss: 0.8666
Epoch [14000], Loss: 0.8666
Epoch [15000], Loss: 0.8695
Epoch [16000], Loss: 0.8690
Epoch [17000], Loss: 0.8706
Epoch [18000], Loss: 0.8679
Epoch [19000], Loss: 0.8666
Epoch [20000], Loss: 0.8682
Epoch [21000], Loss: 0.8681
Epoch [22000], Loss: 0.8661
Epoch [23000], Loss: 0.8673
Epoch [24000], Loss: 0.8659
Epoch [25000], Loss: 0.8715
Epoch [26000], Loss: 0.8647
Epoch [27000], Loss: 0.8670
Epoch [28000], Loss: 0.8697
Epoch [29000], Loss: 0.8688
Epoch [30000], Loss: 0.8666
Epoch [31000], Loss: 0.8679
Epoch [32000], Loss: 0.8704
Epoch [33000], Loss: 0.8682
Epoch [34000], Loss: 0.8670
Epoch [35000], Loss: 0.8664
Epoch [36000], Loss: 0.8673
E

KeyboardInterrupt: 

In [37]:
torch.save(GSE25066_AutoEncoder_Model.state_dict(), 'GSE25066_AE_0.pth')

# 生成对抗网络模型训练

In [9]:
from torch.utils.data import DataLoader
from torch.optim import RMSprop
import pandas as pd

# 加载数据
data = torch.tensor(df.values, dtype=torch.float32)
data_loader = DataLoader(data, batch_size=32, shuffle=True)

# 初始化模型
generator = Generator(input_dim=100, output_dim=13236).to(device)
discriminator = Discriminator().to(device)

# 初始化优化器
gen_optimizer = RMSprop(generator.parameters(), lr=0.0001)
dis_optimizer = RMSprop(discriminator.parameters(), lr=0.0001)



[100, 8192, 4096, 2048, 1024, 512, 256, 128]
[13236, 8192, 4096, 2048, 1024, 512, 256, 128]


In [10]:
generator

Generator(
  (net): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=1024, out_features=2048, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Linear(in_features=2048, out_features=4096, bias=True)
    (9): LeakyReLU(negative_slope=0.2)
    (10): Linear(in_features=4096, out_features=8192, bias=True)
    (11): LeakyReLU(negative_slope=0.2)
    (12): Linear(in_features=8192, out_features=13236, bias=True)
    (13): Tanh()
  )
)

In [11]:
discriminator

Discriminator(
  (net): Sequential(
    (0): Linear(in_features=13236, out_features=8192, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=8192, out_features=4096, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=4096, out_features=2048, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=2048, out_features=1024, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Linear(in_features=1024, out_features=512, bias=True)
    (9): LeakyReLU(negative_slope=0.2)
    (10): Linear(in_features=512, out_features=256, bias=True)
    (11): LeakyReLU(negative_slope=0.2)
    (12): Linear(in_features=256, out_features=128, bias=True)
    (13): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [12]:
# 训练模型
train(device, generator, discriminator, gen_optimizer, dis_optimizer, data_loader, epochs=200)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x13236 and 128x256)